# Implementing a Key-Value Database
In this short project, we will be using the b-tree data structure as the building block for a fully functioning, saving-to-disk, key-value store. 

This project will have a data engineering focus and thus different data structures and efficiency will be key. For an analysis focused project, look for some database projects in the Data Analysis Github repo.

## Introduction
A key-value store is a database that operates similar to a Python dictionary but stored in a file. There are the important get and set methods that the dictionary contains, but the key-value store will also provide an API for saving to disk, loading from disk, range queries of our data, and others.

Our goal will be to create an easy to use, flexible, and adaptable key value store that other developers could use in their projects.

There are multiple implementations of the key-value store that are used in production grade systems worldwide. Some example open source implementations are Redis, CouchDB, Mongo, and Cassandra (which uses a b-tree as the underlying datastructure). These are just a few of the major projects that implement a key-value store that is similar to the one we will be creating.

Be careful though; just because we have the same underlying data structure does not mean that our project is production ready. These types of projects have to cater to multiple use cases, are robust and well tested, but the most important is a well defined and easy to use API. In order to be in the same tier, our project has to most certainly have good API design, and we won't get there building a general case. This is why a perfect "sample" project does not exist, and most times we must build to cater to specific use cases.

With that being said, for this project, we will inherit the majority of the behavior from the BTree structure. However, we will redesign some of the API to make it more user friendly. Finally, we will be implementing some additional behavior to make it function as the key-value database that we expect.

We will note that the project itself will be very short in code, and this is not a bad thing! Good design can often very well result in small code, which makes it easy to read, and easy to use. Because this project has a data engineering focus, this is exactly what we want.

## Implementing the Get and the Set
Every key-value store (abbreviated to kv store) implements a get() and set() method that serve as its most important functionality. Like a Python dict, the kv store will set a value for some given key only if value is not None. However, the set() method should also function different from a Python dictionary by requiring the uniqueness of the key, that means if the key exists in the kv store, then throw a duplicate error.

For the get() method, we also want to replicate the same behavior from the Python dictionary. If we wish to get() the value for some given key, then we should get the saved value. However, if that key does not exist, the key-value store should throw an exception.

An important thing to remember is to always think about the API design as we develop the get() and set() methods. A handy class to use is the NodeKey class that is attached to the project file. With this class, we try to make it so the user of the kv store does not have to create a NodeKey objects first before calling set() on the kv store. Below is the first implementation of the project:

In [1]:
# get pickle
# documentation: https://docs.python.org/3/library/pickle.html
# import stuff from btree file
import pickle
from btree import Node, BTree, NodeKey

# make a KV class that inherits BTree
class KV(BTree):
    # empty init TO BE CONTINUED
    def __init__(self, values=None):
    
    # get method
    def get(self, key):
        # search for a key given
        value = self.search(self.root, key)
        # simply return the value or an error if there is none
        if value is None:
            raise KeyError('There is no value for key "{}"'.format(key))
        return value
    
    # set method
    def set(self, key, value):
        # if no value given - obv error
        if value is None:
            raise ValueError('Cannot store None values')
        # key must match type
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type {}'.format(self.type))
        # set exists to make sure it is none
        exists = self.search(self.root, key)
        if exists is not None:
            raise ValueError('Cannot store duplicate key values')
        
        # insert the node where the key is given value
        node = NodeKey(key, value)
        self.insert(node)

We can see that a we have a pretty good representation of the KV class going now, starting with the get and set. This is actually a majority of the work already, but we can still do a few more things.

## Override the Initializer
Right now we allow any type to be set as our key. For our kv store to work with a b-tree, we need to set some data type as our key. If we don't then we would be doing comparisons on ints and strs for example. We should override the initializer and set some type that we use to validate the inserted keys in our set() method.

We can verify a type by using this Python standard library function:

In [4]:
# examples of isinstance
isinstance('hello', str)  # True
isinstance('hello', int)  # False
isinstance([], list)      # True

True

An important thing to note is that when overriding the __init__() method, we also want to also call the parent __init__() method. If we don't then we would not be able to set the self.t and self.root properties that are expected. To call the parent __init__() method, we use a function named super() which is just another way of writing the BTree class:

In [5]:
# example phony init method
def __init__(self, some_arg, another_arg):
    # Pass some `some_arg` and `another_arg` to the
    # parent `__init__` method.
    super().__init__(some_arg, another_arg)

Below we copy and paste the class code from above to implement these changes. The final piece of code itself will not be very long so we will continue to do this with each step. This can be good as they can be viewed side by side to see where each change happened.

In [6]:
# get pickle
# documentation: https://docs.python.org/3/library/pickle.html
# import stuff from btree file
import pickle
from btree import Node, BTree, NodeKey

# make a KV class that inherits BTree
class KV(BTree):
    # adjust init to override - sets data type and degree to 10
    def __init__(self, type_, values=None):
        self.type = type_
        # uses super to call origninal
        super().__init__(10)
    
    # get method
    def get(self, key):
        # search for a key given
        value = self.search(self.root, key)
        # simply return the value or an error if there is none
        if value is None:
            raise KeyError('There is no value for key "{}"'.format(key))
        return value
    
    # set method
    def set(self, key, value):
        # if no value given - obv error
        if value is None:
            raise ValueError('Cannot store None values')
        # key must match type
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type {}'.format(self.type))
        # set exists to make sure it is none
        exists = self.search(self.root, key)
        if exists is not None:
            raise ValueError('Cannot store duplicate key values')
        
        # insert the node where the key is given value
        node = NodeKey(key, value)
        self.insert(node)

It is noted that we already used a case of isinstance once in the Set method to account for key errors. Now, the __init__() method can be properly overridden to be called for different data types.

## Reimplementing the Range Queries
The current range queries for greater_than() and less_than() are a bit tedious to work with (in the btree file). First, we have to pass in the root node, then we have to pass in a value, an optional upper bound, and then an inclusive! All these can be frustrating for a user who just wants to do a simple range query on their data.

To solve this user experience problem, we will implement a new range query method that will abstract away the tedious work for the user. We will still use the old range queries under the hood, but we provide the user with better error returns so they can iteratively learn how to use the range query. This is how we would like the method to behave like:

In [7]:
# this should return all the keys that are within the
# interval (0, 5) inclusive.
kv.range_query([0, 5], inclusive=True)
# This should return all values greater than 6.
kv.range_query([6, None])
# This should return all values less than 6.
kv.range_query([None, 6])

NameError: name 'kv' is not defined

Note: the code above will not run without a kv instance defined first. It is only used as an example.

The main thing and condition here for the range query is that the user should not have to pass in the root node, it's too tedious to remember every time.

In [8]:
# get pickle
# documentation: https://docs.python.org/3/library/pickle.html
# import stuff from btree file
import pickle
from btree import Node, BTree, NodeKey

# make a KV class that inherits BTree
class KV(BTree):
    # adjust init to override - sets data type and degree to 10
    def __init__(self, type_, values=None):
        self.type = type_
        # uses super to call origninal
        super().__init__(10)
    
    # get method
    def get(self, key):
        # search for a key given
        value = self.search(self.root, key)
        # simply return the value or an error if there is none
        if value is None:
            raise KeyError('There is no value for key "{}"'.format(key))
        return value
    
    # set method
    def set(self, key, value):
        # if no value given - obv error
        if value is None:
            raise ValueError('Cannot store None values')
        # key must match type
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type {}'.format(self.type))
        # set exists to make sure it is none
        exists = self.search(self.root, key)
        if exists is not None:
            raise ValueError('Cannot store duplicate key values')
        
        # insert the node where the key is given value
        node = NodeKey(key, value)
        self.insert(node)
    
    # range query method - optional inclusive
    def range_query(self, interval, inclusive=False):
        # error message if not list or tuple
        if not isinstance(interval, (list, tuple)) and len(interval) != 2:
            raise ValueError('The first argument must be a list or tuple of length 2')
        
        # make interval
        lower, upper = interval
        
        # if lower is none, do less than method, else greater than - simple for user!
        if lower is None:
            return self.less_than(self.root, upper, inclusive=inclusive)
        return self.greater_than(self.root, lower, upper_bound=upper, inclusive=inclusive)

It is always helpful to raise some hintful errors like the one inthe range_query method. We can obviously go overboard and raise too many trival ones, but a few here and there can be extremely useful to users.

## Dump and Load the KV Store
Now it's time to get to the bread and butter of the kv store, saving and loading it to disk. The behavior we want to provide is to let a user pass in a filename to load or save to the object to disk. To make it easier for the user, we will attach a our filetype name to the end and check if that file is available to read or write out to.

**However, one thing to keep in mind is that we want to provide a load() function that is not an instance method of the KV class. The reason being is that we want to load from a file to create a KV instance. If it was an instance method, then it would be defeating the purpose of the behavior it is trying to provide.**

This is the behavior we would like to see:

In [9]:
# dumps out to a file called 'sample_kv.db'
kv.dump('sample_kv')
# loads from a file called 'sample_kv.db' and
# assigns the loaded object to an instance using
# a function and not a instance method.
kv_loaded = load_kv('sample_kv')

NameError: name 'kv' is not defined

Note: the code above will not run without a kv instance defined first. It is only used as an example.

We implement these methods in the code below.

In [10]:
# get pickle
# documentation: https://docs.python.org/3/library/pickle.html
# import stuff from btree file
import pickle
from btree import Node, BTree, NodeKey

# make a KV class that inherits BTree
class KV(BTree):
    # adjust init to override - sets data type and degree to 10
    def __init__(self, type_, values=None):
        self.type = type_
        # uses super to call origninal
        super().__init__(10)
    
    # get method
    def get(self, key):
        # search for a key given
        value = self.search(self.root, key)
        # simply return the value or an error if there is none
        if value is None:
            raise KeyError('There is no value for key "{}"'.format(key))
        return value
    
    # set method
    def set(self, key, value):
        # if no value given - obv error
        if value is None:
            raise ValueError('Cannot store None values')
        # key must match type
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type {}'.format(self.type))
        # set exists to make sure it is none
        exists = self.search(self.root, key)
        if exists is not None:
            raise ValueError('Cannot store duplicate key values')
        
        # insert the node where the key is given value
        node = NodeKey(key, value)
        self.insert(node)
    
    # range query method - optional inclusive
    def range_query(self, interval, inclusive=False):
        # error message if not list or tuple
        if not isinstance(interval, (list, tuple)) and len(interval) != 2:
            raise ValueError('The first argument must be a list or tuple of length 2')
        
        # make interval
        lower, upper = interval
        
        # if lower is none, do less than method, else greater than - simple for user!
        if lower is None:
            return self.less_than(self.root, upper, inclusive=inclusive)
        return self.greater_than(self.root, lower, upper_bound=upper, inclusive=inclusive)
    
    # dump method that saves to a file
    def dump(self, filename):
        # concat .db
        filename = filename + '.db'
        # pickle dump method and return True to show it worked, else False
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
            return True
        return False
    
    # immutable via inheritance - load method
    @staticmethod
    def load(filename):
        # concat .db
        filename = filename + '.db'
        # pickle load to this file!
        with open(filename, 'rb') as f:
            return pickle.load(f)

There are a few extra things in th code such as @staticmethod, but hopefully a well commented code will help readers understand. The dump and load methods look obviously very similar as their processes are basically the same; their end goals are just different.

## Load from Dictionary
One last thing to do. To make things just a bit easier, we will write a method to load from a dict that loads in keys and values from a Python dict. We must be aware that a dict can have any key and we place a restriction on our keys.

This makes it so a user can not only load from file but from any Python dict as well.

In [13]:
# get pickle
# documentation: https://docs.python.org/3/library/pickle.html
# import stuff from btree file
import pickle
from btree import Node, BTree, NodeKey

# make a KV class that inherits BTree
class KV(BTree):
    # adjust init to override - sets data type and degree to 10
    def __init__(self, type_, values=None):
        self.type = type_
        # uses super to call origninal
        super().__init__(10)
    
    # get method
    def get(self, key):
        # search for a key given
        value = self.search(self.root, key)
        # simply return the value or an error if there is none
        if value is None:
            raise KeyError('There is no value for key "{}"'.format(key))
        return value
    
    # set method
    def set(self, key, value):
        # if no value given - obv error
        if value is None:
            raise ValueError('Cannot store None values')
        # key must match type
        if not isinstance(key, self.type):
            raise KeyError('Key must be of type {}'.format(self.type))
        # set exists to make sure it is none
        exists = self.search(self.root, key)
        if exists is not None:
            raise ValueError('Cannot store duplicate key values')
        
        # insert the node where the key is given value
        node = NodeKey(key, value)
        self.insert(node)
    
    # range query method - optional inclusive
    def range_query(self, interval, inclusive=False):
        # error message if not list or tuple
        if not isinstance(interval, (list, tuple)) and len(interval) != 2:
            raise ValueError('The first argument must be a list or tuple of length 2')
        
        # make interval
        lower, upper = interval
        
        # if lower is none, do less than method, else greater than - simple for user!
        if lower is None:
            return self.less_than(self.root, upper, inclusive=inclusive)
        return self.greater_than(self.root, lower, upper_bound=upper, inclusive=inclusive)
    
    # dump method that saves to a file
    def dump(self, filename):
        # concat .db
        filename = filename + '.db'
        # pickle dump method and return True to show it worked, else False
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
            return True
        return False
    
    # immutable via inheritance - load method
    @staticmethod
    def load(filename):
        # concat .db
        filename = filename + '.db'
        # pickle load to this file!
        with open(filename, 'rb') as f:
            return pickle.load(f)
    
    # load from dict method that takes in dictionary
    def load_from_dict(self, dictionary):
        # set method for each key and value, simple
        for key, value in dictionary.items():
            self.set(key, value)

Now our whole class is complete! Obviously we can add more methods if we wanted, but the basis of what we have now can be perfectly fine for a user. 

To test our class out now, let's do some basic executions!

In [18]:
# initialize an instance
db = KV(int)

# do some sets
db.set(1, 'hello')
db.set(2, 'world')
db.set(3, 'this')
db.set(4, 'is')

# see what we got so far
print(db.range_query([1,3]))
print()

# save it into sample_store file
db.dump('sample_store')

# load it back into kv instance
kv = KV.load('sample_store')

# see what we got again (should be same)
print(kv.range_query([1,3]))
print()

# add some additional keys
additional_keys = {
    5: 'a',
    6: 'simple',
    7: 'kv store'
}

# load from the dict these keys
kv.load_from_dict(additional_keys)

# see them in action
print(kv.range_query([4,8]))
print()

# test get method
print(kv.get(5))

[<NodeKey: (2, world)>]

[<NodeKey: (2, world)>]

[<NodeKey: (5, a)>, <NodeKey: (6, simple)>, <NodeKey: (7, kv store)>]

a


Looks like everything works great! The example were really easy to follow, and even the print statements show exactly what we are doing. It looks like this basic database project was a success!

## Further Analysis / Next Steps
Even though we said our "store" was a success, it never really is complete, is it? This isn't a knock on our project, more so recognizing that most projects are never complete. There are a lot more behaviors that you could implement to make it even more useful. 

These are often use case specific though and often times implementing a method that ends up not being very helpful due to circumstances is just a waste. Below, we will just highlight some of the different things that are **possible** to be implemented, should they ever be needed! 

Note: While not a KV database, for a complete database instanced project with full analysis, check out the "Baseball" project in the Data Analysis Github repo.

## Possible Features
A few additional features you could provide:

* Enhancing the API and making it look more "Pythonic" using special methods for set and get. (This is really more of a aesthetic thing rather than a efficiency thing - but often times better looking code can still result in the latter.)

* A different serializer so that you can use different versions of Python. (Something like JSON might work here.)

* A delete method that would remove nodes from the kv store. (This is an efficiency improvement that would work if you needed something that was more flowing. Instead of dumping and loading all at once, perhaps you are in need of just removing a couple nodes. This will essentially work similar but opposite to a set.)

* Saving out to multiple files using the keys as pointers to files for bigger databases. (This would be a good use if your file ends up being too big (ours obviously isn't). The code will have the same foundation except now you will need to obviously insert more than one file name and the db will be split in certain areas.)

* Implementing the kv store on top of a B+Tree, another variation of the b-tree family of data structures. (This is one of those different strokes for different folks type situations - the BTree works fine, but a B+Tree might be better depending on your preference and situation. B+Tree: https://en.m.wikipedia.org/wiki/B%2B_tree.)

Remember that all these are just a sample of the things that you can do with the kv store. The possibilities are literally endless, and if your situation calls for it, more likely than not, you can do it!

It is also important to note that Python itself and its primitive libraries should not be underestimated. Open source code is open source for a reason - for the best minds to work on and improve things. If a method, function, or class already exists that you are trying to add, it's probably best to not re-invent the wheel.

## Conclusion
In this project, we highlighted the inner workings of a KV key-value database class that used attributes of a BTree. We used helper code that can be viewed in their respective files within this directory. We also added various methods such as a getter, setter, load, dump, range_query, etc. to the class that proved to be very helpful when a user is writing code. 

Depending on your situation and what you need, this code can actually be used as a basis for an actual use case for a specific KV database that you need in industry. In a truly individualized and customized database, the code will be very similar to many other classes, but never the same. This is a blessing that must be recognized; often times improving on existing work comes from small tweaks here and there, but the foundation stays the same. Complete overhauls are rarely necessary.

Overall, this project was a very nice showing of how basic b-trees work and a basic key-value database in action. A b-tree is one of the more complex data structures, but once mastered, it can seem as basic as anything else. As far as data engineering goes, like it is always said, practice makes perfect.